Lenses and Lens Lists
=========================

Lenses are obviously an integral component in every mode matching setup.
Beam Corset can model lenses as infinitely thin using the :class:`~corset.core.ThinLens` class or with finite thickness using the :class:`~corset.core.ThickLens` class. Collections of lenses can saved loaded and managed using the :class:`~corset.database.LensList` class.

Thin Lenses
-----------

The simplest lens model in Beam Corset is the thin lens, used to model lenses which are significantly thinner than their focal length. Thin lenses are simply specified directly by their focal. In addition to that, we can also specify physical margins to account for the physical size of the lens and prevent overlaps. Finally, we can also give them a distinct name that will show up in plots and can be used to refer to them in lens lists. Thin lenses are modeled using the :class:`~corset.core.ThinLens` class.

In [ ]:
from corset import ThinLens

thin_lens = ThinLens(200e-3) # margins default to 0
thin_lens_all_args = ThinLens(focal_length=100e-3, left_margin=5e-3, right_margin=5e-3, name="TL100")
print(repr(thin_lens))
print(repr(thin_lens_all_args))

The left and right margins are relative to the focal plane of the lens. They may be negative individually but they must add up to at least zero meaning that all lenses can not occupy negative space.

The margins and name are also shown in optical setup plots.

In [ ]:
from corset import Beam, OpticalSetup

beam = Beam.from_gauss(focus=0, waist=200e-6, wavelength=1064e-9)
setup = OpticalSetup(beam, [(0.1, thin_lens), (0.2, thin_lens_all_args)])
setup

Lenses without a name are labeled with their focal length in millimeters.

Thick Lenses
------------

Lenses may also be modeled as thick lenses, in this case we specify them by their input and output radius of curvature, their refractive index and their thickness. The signs of the radius of curvature follow the standard optics convention where positive radii correspond to surfaces that are convex when viewed from the input side of the lens. This means that a biconvex lens will have a positive input radius of curvature and a negative output radius of curvature.

Like thin lenses, we can also give thick lenses physical margins and names. The physical margins extend from the plane centered between the two surfaces of the lens and are independent from the optical thickness.

Since the beam propagation model requires that the beam exits an element in the same axial position as it entered it, the ray transfer matrices for thick lenses include a negative amount of free space propagation before and after the lens. This way the sum of all propagation matrices that the element is made up of, is zero. One notable consequence of this treatment is that the calculated and plotted beam radius is not accurate for axial positions that are inside thick lenses. This also means that there may be a small discontinuity in beam radius across thick lenses in the plotted beam profiles.

Thick lenses are modeled using the :class:`~corset.core.ThickLens` class. For convenience and interoperability with thin lenses, the thick lens class also has a property :attr:`~corset.core.ThickLens.focal_length` that yields the approximate focal length calculated using the `lensmaker's equation <https://en.wikipedia.org/wiki/Lens#Lensmaker's_equation>`_. Thick lenses cannot be described using a single focal length so this value should only be used as an estimate and not for calculations.

In [ ]:
from corset import ThickLens

thick_lens = ThickLens(in_roc=50e-3, out_roc=-50e-3, thickness=5e-3, refractive_index=1.5)
print(repr(thick_lens))
print(f"{thick_lens.focal_length = }")

We can create flat interfaces by setting an infinite radius of curvature, i.e. `float('inf')` or `np.inf`. However, instead of using infinity directly, you should use the alias :attr:`ThickLens.FLAT <corset.core.ThickLens.FLAT>` to improve readability and convey intent. 

In [ ]:
plano_convex = ThickLens(ThickLens.FLAT, 50e-3, 10e-3, 1.5)
print(repr(plano_convex))

Lens Lists
----------

Since you will likely use the same set of lenses for most setups, it makes sense to build a database of these lenses to avoid having to look up and specify their parameters every time you want to use them. For this, we can use a :class:`~corset.database.LensList` which is a list of lenses with some extra convenience features.

In [ ]:
from corset import LensList

my_lenses = LensList([
    ThinLens(50e-3, name="TL50"),
    ThinLens(100e-3, name="TL100"),
    ThickLens(ThickLens.FLAT, -100e-3, 10e-3, 1.5, name="PC200"),
    ThickLens(300e-3, -300e-3, 5e-3, 1.5, name="BC300"),
])
my_lenses

Lens lists will display as a :class:`~pandas.DataFrame` in Jupyter notebooks, showing the lenses parameters. To allow both thin and thick lenses in the same list, the DataFrame will show `NaN` for parameters that do not apply to thin lenses, and display the approximate focal length calculated using the lensmaker's equation for thick lenses.

Lens lists can be saved to and loaded from CSV files using the :meth:`~corset.database.LensList.save_csv` and :meth:`~corset.database.LensList.load_csv` methods. These forward to :meth:`pandas.DataFrame.to_csv` and :func:`pandas.read_csv` respectively, so they accept the same kinds of inputs like URLs to easily load lens lists from online repositories.

In [ ]:
from IPython.display import Pretty

my_lenses.save_csv("my_lenses.csv")
Pretty("my_lenses.csv")

When loading lenses from CSV files, the lens type is determined from the type filed and must be consistent with the parameters specified.

In [ ]:
LensList.load_csv("my_lenses.csv") == my_lenses

In addition to normal scalar indexing, we can also index into the list lens name. To ensure that this is always unambiguous, lens names must be unique within a lens list. This is enforced by the constructor.

In [ ]:
print(repr(my_lenses[0]))
print(repr(my_lenses["PC200"]))

We can easily create a subset of a lens list by indexing with a list of names or indices.

In [ ]:
my_lenses[[0, 3]]

In [ ]:
my_lenses[["TL50", "BC300"]]

Lens lists can be concatenated with other lens lists or normal lists using the `+` operator.

In [ ]:
new_list = my_lenses + [ThinLens(focal_length=25e-3, name="TL25")]
new_list